<a href="https://colab.research.google.com/github/mendyfba/ML-Promoter-Classification/blob/main/classification_promoteurs_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importation des bibliothèques

In [314]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


2. Chargement du dataset

In [315]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/promoter-gene-sequences/promoters.data"

In [316]:
df= pd.read_csv(url,header=None, names = ["Class", "ID", "Sequence"])

In [317]:
print(df[["Sequence", "ID" , "Class"]].head(5))

                                            Sequence         ID Class
0  \t\ttactagcaatacgcttgcgttcggtggttaagtatgtataat...        S10     +
1  \t\ttgctatcctgacagttgtcacgctgattggtgtcgttacaat...       AMPC     +
2  \t\tgtactagagaactagtgcattagcttatttttttgttatcat...       AROH     +
3  \taattgtgatgtgtatcgaagtgtgttgcggagtagatgttagaa...      DEOP2     +
4  \ttcgataattaactattgacgaaaagctgaaaaccactagaatgc...  LEU1_TRNA     +


3. Nettoyage et prétraitement

In [318]:
df['Sequence'] = df['Sequence'].astype(str).apply(lambda x: x.replace('\t', '').replace(' ', ''))

4. Tokenization par K-mers

In [319]:
def get_kmers(seq,k=6):
  seq = seq.replace(" ", "").replace("\n", "").replace("\r", "").upper()
  kmers=[]

  for i in range(len(seq)-k+1):
    word=seq[i:i+k]
    kmers.append(word)

  return " ".join(kmers)

df["words"]= df['Sequence'].apply(get_kmers)

In [320]:
print(df[['Sequence', 'words']].head())

                                            Sequence  \
0  tactagcaatacgcttgcgttcggtggttaagtatgtataatgcgc...   
1  tgctatcctgacagttgtcacgctgattggtgtcgttacaatctaa...   
2  gtactagagaactagtgcattagcttatttttttgttatcatgcta...   
3  aattgtgatgtgtatcgaagtgtgttgcggagtagatgttagaata...   
4  tcgataattaactattgacgaaaagctgaaaaccactagaatgcgc...   

                                               words  
0  TACTAG ACTAGC CTAGCA TAGCAA AGCAAT GCAATA CAAT...  
1  TGCTAT GCTATC CTATCC TATCCT ATCCTG TCCTGA CCTG...  
2  GTACTA TACTAG ACTAGA CTAGAG TAGAGA AGAGAA GAGA...  
3  AATTGT ATTGTG TTGTGA TGTGAT GTGATG TGATGT GATG...  
4  TCGATA CGATAA GATAAT ATAATT TAATTA AATTAA ATTA...  


5. Vectorisation (Bag of Words)

In [321]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

cv = CountVectorizer()
counts = cv.fit_transform(df['words'])
labels = df['Class'].map({'+': 1, '-': 0})

In [322]:
df_counts=pd.DataFrame(counts.toarray(), columns=cv.get_feature_names_out())
print(df_counts.iloc[:5, :10])

   aaaaaa  aaaaac  aaaaag  aaaaat  aaaaca  aaaacc  aaaacg  aaaact  aaaaga  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       1       0       0       0   

   aaaagc  
0       0  
1       0  
2       0  
3       0  
4       1  


6. Division des données

In [323]:
x_train, x_test, y_train, y_test = train_test_split(counts, labels, test_size=0.2, random_state=42)

In [324]:
print(f"Train : {x_train.shape[0]} séquences")
print(f"Test : {x_test.shape[0]} séquences")

Train : 84 séquences
Test : 22 séquences


7. Entraînement du modèle Random Forest

In [325]:
from sklearn.ensemble import RandomForestClassifier

model= RandomForestClassifier(n_estimators=100, random_state=42)

In [326]:
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

8. Évaluation des performances

In [327]:
score= model.score(x_test, y_test)
print()
print("Précision de l'IA :", round(score * 100, 2), "%")


Précision de l'IA : 95.45 %


9. Module de prédiction interactive

In [328]:
# Test du modèle avec la région promotrice Plac (-150 à +50) de l'opéron lactose d'Escherichia coli (associée au gène lacZ)
#GCGCAATTAATGTGAGTTAGCTCACTCATTAGGCACCCCAGGCTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGAATTGTGAGCGGATAACAATTTCACACAGGAAACAGCTATGACCATGATTACGGATTCACTGGCCGTCGTTTTACAACGTCGTGACTGGGAAAACCCTGGCGTTACCCAACTTAATCGCCTTGCAGCACATCCCCCTTTCGCCAGCTGGCGTAATAG

user_seq = input("Entrez votre séquence d'ADN ici : ")

if len(user_seq) < 100:
    print("La sequence est trop courte pour une analyse fiable.")
else:
    kmers_test = get_kmers(user_seq)
    vecteur_kmers = cv.transform([kmers_test])
    proba = model.predict_proba(vecteur_kmers)

    score = proba[0][1]
    pourcentage = score * 100

    print("-" * 30)


    if score > 0.5:
        print("RÉSULTAT : C'est une séquence promotrice !")
        print(pourcentage, "% de chance")
    else:
        print("RÉSULTAT : Ce n'est pas unes séquence promotrice.")
        print(100 - pourcentage, "% de chance")

    print("-" * 30)

Entrez votre séquence d'ADN ici : GCGCAATTAATGTGAGTTAGCTCACTCATTAGGCACCCCAGGCTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGAATTGTGAGCGGATAACAATTTCACACAGGAAACAGCTATGACCATGATTACGGATTCACTGGCCGTCGTTTTACAACGTCGTGACTGGGAAAACCCTGGCGTTACCCAACTTAATCGCCTTGCAGCACATCCCCCTTTCGCCAGCTGGCGTAATAG
------------------------------
RÉSULTAT : C'est une séquence promotrice !
76.0 % de chance
------------------------------
